In [1]:
import refinitiv.data as rd
import numpy as np
import pandas as pd
import matplotlib as plt
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
import time
rd.open_session()

<refinitiv.data.session.Definition object at 0x7f692f073a30 {name='codebook'}>

In [2]:
ESG_score_high = 70
ESG_score_low = 30
start_year = 2014
end_year = 2024
index = '.PSI'



In [4]:
psi = rd.get_data(index, fields=['TR.CLOSEPRICE.date', 'TR.CLOSEPRICE'], parameters={'SDate': '{}-01-01'.format(start_year), 'EDate': '{}-01-01'.format(end_year), 'Frq':'CQ'})

In [5]:
psi['Rebased'] = psi['Close Price'] * 100 / psi['Close Price'][0]
psi.head()

,Instrument,Date,Close Price,Rebased
0,.PSI,2014-03-31,6428.71,100.0
1,.PSI,2014-06-30,6844.31,106.46475
2,.PSI,2014-09-30,7283.07,113.289758
3,.PSI,2014-12-29,7230.57,112.473109
4,.PSI,2015-03-31,7940.49,123.516071


In [6]:
def getSubset(dFrame, esgpos):
    # filter out the companies with low ESG score at the begining year
    if esgpos == "High":
        dFrame = dFrame[dFrame['ESG Combined Score'] > ESG_score_high]
        
    elif esgpos == "Low":
        dFrame = dFrame[dFrame['ESG Combined Score'] > ESG_score_high]
        
    return dFrame['Instrument'].tolist()

In [7]:
def getDataForYear(year, esgpos):
    print(f'Getting data for: {year}')
    # get index constituents at the begining year
    df = rd.get_data('{}{}({}-01-01)'.format('0#', index, year), fields=['TR.TRESGCScore'], parameters={'SDate': '{}-01-01'.format(year), 'Period': 'FY0'})
    # filter out the instruments based on ESG ratings
    subset = getSubset(df, esgpos)
    # get the performance data for this subset
    df2 = rd.get_data(subset, fields=['TR.CLOSEPRICE.date', 'TR.CLOSEPRICE'], parameters={'SDate': '{}-01-01'.format(year), 'EDate': '{}-01-01'.format(year + 1), 'Frq':'CQ'})
    df2 = df2.dropna()
    # consolidate the price data for instruments
    alphaList = []
    ser = df2['Date'].value_counts()
    for x in ser[ser > 5].index:
        ser = df2.loc[df2['Date'] == x].sum(numeric_only=True)
        alphaList.append({'Date': x, 'Close Price': ser['Close Price']})
    return pd.DataFrame(alphaList)


In [19]:
df_high = pd.DataFrame()
for x in range(start_year, end_year):
    ret = getDataForYear(x, "High")
    df_high = pd.concat([df_high, ret])
    time.sleep(3)
df_high = df_high.sort_values(by=['Date'])
df_high = df_high.reset_index(drop=True)

Getting data for: 2014


RDError: Error code -1 | Backend error. 400 Bad Request Requested universes: ['0#.PSI(2014-01-01)']. Requested fields: ['TR.TRESGCSCORE']

In [ ]:
df_high['Rebased'] = df_high['Close Price'] * 100 / df_high['Close Price'][0]
df_high.head()

In [ ]:
df_high

In [ ]:
df_low = pd.DataFrame()
for x in range(start_year, end_year):
    ret = getDataForYear(x, "Low")
    df_low = pd.concat([df_low, ret])
    time.sleep(3)
df_low = df_low.sort_values(by=['Date'])
df_low = df_low.reset_index(drop=True)
df_low['Rebased'] = df_low['Close Price'] * 100 / df_low['Close Price'][0]
df_low.head()

In [ ]:
psi.JOIN([df_high, df_low], on = "Date", how = "Inner")